In [3]:
import jieba
from jieba import analyse

In [4]:
print('*****案例二********')
content='该课题在搜集柴胡种质资源基础上，开展了生物学特性、分子标记及栽培技术等方面的研究，形成的主要创新性结果如下：（1）通过AFLP、ISSR分子标记和ITS序列分析，明确了11个柴胡种源之间的亲缘关系；综合大田出苗率、抗根腐病能力、柴胡皂苷含量等8个指标，利用主成分分析法筛选出适植华北地区优质柴胡种源5个。（2）研究提出了柴胡栽培关键技术：促进种子萌发的处理为0.5g/L的硫酸锰浸种24h；适宜播期为3月中下旬；适宜播种量为3kg/亩；施肥比例为一年收获N25:P18:K18，二年收获N25:P6:K12；适宜采收期在7月中下旬~8月下旬，制种期为早熟品种八月中上旬、晚熟品种九月中上旬；在此基础上集成形成了柴胡规范化栽培技术体系并形成技术规程。建立了柴胡根腐病病菌检测方法，鉴定出柴胡根腐病病原菌为茄腐镰刀菌（Fusarium solani Will.）。采用“高校+公司+基地+农户”的模式，示范推广柴胡规范化栽培技术面积较大。该研究成果具有新颖性，先进性及实用性。经济效益、社会效益明显，具有良好的生产应用前景'
# 第一个参数：待提取关键词的文本
# 第二个参数：返回关键词的数量，重要性从高到低排序
# 第三个参数：是否同时返回每个关键词的权重
# 第四个参数：词性过滤，为空表示不过滤，若提供则仅返回符合词性要求的关键词,allowPOS('ns', 'n', 'vn', 'v') 地名、名词、动名词、动词
keywords=jieba.analyse.extract_tags(content,topK=5,withWeight=True,allowPOS=())
#访问提取结果
for item in keywords:
    print(item[0],item[1])

Building prefix dict from the default dictionary ...


*****案例二********


Dumping model to file cache C:\Users\GECHEN~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.307 seconds.
Prefix dict has been built successfully.


柴胡 0.5222591670964383
根腐病 0.25255972067465754
栽培技术 0.2186831735753425
N25 0.1637639383958904
种源 0.15887798026027397


In [8]:
#基于 TextRank 算法的关键词抽取
print('*****案例三********')
# 同样是四个参数，但allowPOS默认为('ns', 'n', 'vn', 'v')
# 即仅提取地名、名词、动名词、动词
keywords = jieba.analyse.textrank(content, topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))
# 访问提取结果
for item in keywords:
    print(item)
    # 分别为关键词和相应的权重
#     print(item[0], item[1])

*****案例三********
根腐病
形成
规范化
标记
具有


In [ ]:
"""
LDA主题模型
咱们来用LDA主题模型建模，看看这些新闻主要在说哪些topic。
首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。
[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]
"""
from gensim import corpora, models, similarities
import gensim
"""
载入停用词
"""
stopwords=pd.read_csv("./data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values
 
import jieba
import pandas as pd
df = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
 
sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        #print(segs)
        segs = filter(lambda x:len(x)>1, segs)
        segs = filter(lambda x:x not in stopwords, segs)
        #print(segs)
        sentences.append(list(segs))
    except Exception:
        print(line) 
        continue
        
"""
词袋模型
"""
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]
"""
LDA建模
"""
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)
"""
我们查一下第3号分类，其中最常出现的单词是：
"""
print (lda.print_topic(3, topn=5))
"""
我们把所有的主题打印出来看看
"""
for topic in lda.print_topics(num_topics=20, num_words=8):
    print (topic[1])
 
"""
保存模型
"""
from sklearn.externals import joblib
joblib.dump(lda,"./主题分类1.pkl")